 # Python ML final _ Fuel efficiency prediction with "means of categories"
 
 By 國立陽明交通大學 醫學三 陳冠元 10701054

 
 Inspired by https://www.freecodecamp.org/news/how-to-build-your-first-neural-network-to-predict-house-prices-with-keras-f8db83049159/

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#  Import Data
Import the csv file to have a look about the datasets.

In [ ]:
#Just import the dataset so we can take a look...
import pandas as pd
data = pd.read_csv('../input/co2-emission-by-vehicles/CO2 Emissions_Canada.csv')
data

# Preprocessing
Make and model of the vehicle is removed since it could overwhelm the neural network.

Other fuel consumption data is removed due to the algebraic relationship between fuel consuption and CO2 emission.

In [ ]:
# Cannot quantify Make and Model of an automobile.
data.pop('Make')
data.pop('Model')

#These four values have tight algebraic relationship with the predicted values.
data.pop('Fuel Consumption City (L/100 km)')
data.pop('Fuel Consumption Hwy (L/100 km)')
data.pop('Fuel Consumption Comb (mpg)')
data.pop('CO2 Emissions(g/km)')

#I thought taking these information would increase model accuracy but it did not.
#data.pop('Cylinders')
#data.pop('Transmission')
#data.pop('Fuel Type')

#Save the modified csv file.
data.to_csv('preprocessed_fuel_efficiency.csv', index=False)
pd.read_csv('preprocessed_fuel_efficiency.csv')

Microsoft Excel function for averaging a column according to the condition of another column:

https://www.excelforum.com/excel-general/541631-calculate-average-in-a-column-based-on-criteria-in-another-column.html

Dealing with vehicle class, transmission, fuel type information with means of groups(?)

The means stand for the CO2 emission ot the group in grams per kilometer.

In [ ]:
#Read in the csv file that contain the information about the means of different groups.
means_of_groups = pd.read_csv('../input/means-of-groups/means of groups.csv')

#Convert it into an array
MOG = means_of_groups.values
means_of_groups

#What I really did is use Microsoft Excel to calculate the average CO2 emission according to the attributions below:

According to Environmental Protection Agency of US, 
Average fuel efficiency of typical passenger vehicle is 10.69L/100km

https://www.epa.gov/greenvehicles/greenhouse-gas-emissions-typical-passenger-vehicle

My old method of replacing attribution information with integers, but it someimes cause problems.

In [ ]:
#new_data = pd.read_csv("preprocessed_fuel_efficiency.csv")

#x == 0
#for x in range(len(MOG)):
   # new_data['Vehicle Class'] = new_data['Vehicle Class'].replace({str(MOG[x][0]): MOG[x][1]})
   # new_data['Transmission'] = new_data['Transmission'].replace({str(MOG[x][0]): MOG[x][1]})
    #new_data['Fuel Type'] = new_data['Fuel Type'].replace({str(MOG[x][0]): MOG[x][1]})
   # x=x+1    

#new_data.to_csv("preprocessed_fuel_efficiency.csv", index=False)

#new_data

In [ ]:
import numpy as np

new_data = pd.read_csv("preprocessed_fuel_efficiency.csv")
dataset = new_data.values

dataset

https://stackoverflow.com/questions/18716564/python-cant-assign-to-literal

https://www.guru99.com/variables-in-python.html

In [ ]:
# This loop replaces the attribution information with integers.
# Somehow if I go a=0, b=0 python will give me error messages...
a,b,c = 0,0,0

for a in range(len(dataset)):
    for b in range(len(MOG)):
        for c in range(len(dataset[a])):
            if dataset[a][c] == MOG[b][0]:
                dataset[a][c] = MOG[b][1]
            c=c+1
        b=b+1
    a=a+1    

#This loop simplifies the fuel efficiency to 0 = below average; 1 = above average. 
x,y=0,0
for x in range(len(dataset)):
    if dataset[x][5] < 10.69:
        dataset[x][5] = 1
        y=y+1
    else:
        dataset[x][5] = 0
    x=x+1
z=y/x

#Just to examine the percentage of car models that perform above average in fuel efficiency.
#If the results turn out to be near 0.5, it means the dataset coresponds to EPA data.
print(z)
#See how the attribution information and fuel efficiency data is modified.
dataset

In [ ]:
#The reason why have to save the array back to csv file is that the type of the array is "object"
#This will cause the data for validation not being accepted by the deep learning model.

#Somehow the problem could be resolved by saving the array to csv and converting it back to array.
#Heading will be lost therefore it had to be added back manually.
dataset_csv = np.asarray(dataset)
np.savetxt("preprocessed_fuel_efficiency.csv", dataset_csv, delimiter=",", 
           header='Mean emission of Vehicle Class, Engine Size(L), Cylinders, Mean emission of Transmission, Mean emission of Fuel Type, Above average fuel efficiency')
new_data = pd.read_csv("preprocessed_fuel_efficiency.csv")

#This is the dataset AFTER PREPROCESSING.
new_data

Splitting the fuel efficiency results into a separated list.

In [ ]:
#Converting the preprocessed csv file to array...again.
dataset = new_data.values

#Splitting the results
X = dataset[:,0:5]
Y = dataset[:,5]

print(X)
print(Y)

#this way there is a dot after the every element.

In [ ]:
#Make the values of every column between 0 and 1 so the deep learning model can process it.
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
X_scale

In [ ]:
#Since N=7385, 10% for testing and validation is quite enough.
from sklearn.model_selection import train_test_split
X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X, Y, test_size=0.1)
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)
print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

# Deep Learning Model

https://keras.io/api/layers/activations/

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
#5 inputs with 4 hidden neuron layers and 1 output.
#5 inputs with 4 hidden neuron layers and 1 output.
model = Sequential([
    Dense(5, activation='relu', input_shape=(5,)),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [ ]:
#adam optimizer works somehow works better than sgd.
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['AUC','accuracy'])

In [ ]:
#I find 32 per batch is optimal and at least 100 epochs.
hist = model.fit(X_train, Y_train,
          batch_size=32, epochs=100,
          validation_data=(X_val, Y_val))

# Model Evaluation

In [ ]:
model.evaluate(X_test, Y_test)[1]
print(X_test)
print(Y_test)

# Visualization

In [ ]:
#just plot the stuff above out.
import matplotlib.pyplot as plt
plt.plot(hist.history['val_loss'])
plt.plot(hist.history['loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Val', 'Train'], loc='upper right')
plt.show()

In [ ]:
plt.plot(hist.history['val_auc'])
plt.plot(hist.history['auc'])
plt.plot(hist.history['val_accuracy'])
plt.plot(hist.history['accuracy'])
plt.title('Model AUC and Accuracy')
plt.ylabel('AUC and Accuracy')
plt.xlabel('Epoch')
plt.legend(['Val AUC', 'Train AUC','Val Accuracy', 'Train Accuracy'], loc='lower right')
plt.show()